In [2]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.3 MB/s eta 0:00:00


In [3]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 5.3 MB/s eta 0:00:00


In [4]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 20.3 MB/s eta 0:00:00


In [5]:
!pip install pyswarms imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.7 MB/s eta 0:00:00


In [6]:
pip install pyswarms

In [7]:
import pandas as pd
import numpy as np
import pyswarms as ps
import time
import nltk
import warnings
warnings.filterwarnings("ignore")
import optuna
import string
import re
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from skopt.space import Integer, Real, Categorical
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from scipy.stats import loguniform
from scipy.stats import uniform
from scipy.stats import randint
from scipy.stats import uniform, uniform as uniform_real
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

In [8]:
# Download required NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [10]:
df = pd.read_excel('Data Pakai Label.xlsx')
df

,userName,content,score,reviewCreatedVersion,at,Sentimen (Aurel),Sentimen (Ade),Sentimen (Eky),Sentimen Akhir
0,Made Darmawan,Saya kasi bintang 1 dulu ya. Tolong diperbaiki...,1,4.3.0,2023-01-22 12:04:08,-1.0,-1.0,-1.0,Negatif
1,windi tripani,"Apknya memudahkan dalam perubahan data, ngecek...",5,4.3.0,2023-01-27 06:56:51,1.0,1.0,1.0,Positif
2,PURLAMBANG,Saya mau merubah data KIS saya di aplikasi jkn...,1,4.3.0,2023-01-27 09:16:17,-1.0,-1.0,-1.0,Negatif
3,Nanang WR,"Faskes tidak bisa di search, dan yang ditampil...",1,4.3.0,2022-12-28 01:40:56,-1.0,-1.0,-1.0,Negatif
4,Oktriyani Putri,Kenapa ketika ingin masuk akun ada pendaftaran...,1,4.3.0,2023-01-10 02:22:07,-1.0,-1.0,-1.0,Negatif
...,...,...,...,...,...,...,...,...,...
10793,trijanto trijanto,👍👍👍,5,4.2.3,2022-09-28 03:35:58,1.0,1.0,1.0,Positif
10794,emelianus anto,👍👍👍,5,4.2.3,2022-09-28 23:59:06,1.0,1.0,1.0,Positif
10795,Sunar Sunar,🥰🥰🥰,5,4.2.3,2022-09-27 06:52:54,1.0,1.0,1.0,Positif
10796,Sapri Geta,👍👍👍👍👍,5,4.2.3,2022-09-21 15:20:58,1.0,1.0,1.0,Positif


In [11]:
# hapus tanda baca
def remove_punct(tweet):
    tweet = re.sub('[^a-zA-Z0-9 ]', ' ', str(tweet))
    tweet = re.sub('[0-9]+', ' ', tweet)
    tweet = re.sub(r'#', '', str(tweet))
    tweet = re.sub(r'http\S+', ' ', tweet)
    return tweet

df['TEXT'] = df['content'].apply(lambda x:remove_punct(x))

# case folding
df['case_folding'] = df['TEXT'].str.lower()

# tokenisasi
def tokenization(tweet):
    tweet = re.split('\W+', tweet)
    return tweet
df['Tokenization'] = df['TEXT'].apply(lambda x: tokenization(x.lower()))
df.head(10)

# hapus stopwords
stopword = nltk.corpus.stopwords.words('indonesian')

def remove_stopwords(tweet):
    tweet = [word for word in tweet if word not in stopword]
    return tweet

df['Stopword_Removal'] = df['Tokenization'].apply(lambda x: remove_stopwords(x))
df.head(10)


,userName,content,score,reviewCreatedVersion,at,Sentimen (Aurel),Sentimen (Ade),Sentimen (Eky),Sentimen Akhir,TEXT,case_folding,Tokenization,Stopword_Removal
0,Made Darmawan,Saya kasi bintang 1 dulu ya. Tolong diperbaiki...,1,4.3.0,2023-01-22 12:04:08,-1.0,-1.0,-1.0,Negatif,Saya kasi bintang dulu ya Tolong diperbaiki...,saya kasi bintang dulu ya tolong diperbaiki...,"[saya, kasi, bintang, dulu, ya, tolong, diperb...","[kasi, bintang, ya, tolong, diperbaiki, kemamp..."
1,windi tripani,"Apknya memudahkan dalam perubahan data, ngecek...",5,4.3.0,2023-01-27 06:56:51,1.0,1.0,1.0,Positif,Apknya memudahkan dalam perubahan data ngecek...,apknya memudahkan dalam perubahan data ngecek...,"[apknya, memudahkan, dalam, perubahan, data, n...","[apknya, memudahkan, perubahan, data, ngecek, ..."
2,PURLAMBANG,Saya mau merubah data KIS saya di aplikasi jkn...,1,4.3.0,2023-01-27 09:16:17,-1.0,-1.0,-1.0,Negatif,Saya mau merubah data KIS saya di aplikasi jkn...,saya mau merubah data kis saya di aplikasi jkn...,"[saya, mau, merubah, data, kis, saya, di, apli...","[merubah, data, kis, aplikasi, jkn, layar, per..."
3,Nanang WR,"Faskes tidak bisa di search, dan yang ditampil...",1,4.3.0,2022-12-28 01:40:56,-1.0,-1.0,-1.0,Negatif,Faskes tidak bisa di search dan yang ditampil...,faskes tidak bisa di search dan yang ditampil...,"[faskes, tidak, bisa, di, search, dan, yang, d...","[faskes, search, ditampilkan, sesuai, gps, lha..."
4,Oktriyani Putri,Kenapa ketika ingin masuk akun ada pendaftaran...,1,4.3.0,2023-01-10 02:22:07,-1.0,-1.0,-1.0,Negatif,Kenapa ketika ingin masuk akun ada pendaftaran...,kenapa ketika ingin masuk akun ada pendaftaran...,"[kenapa, ketika, ingin, masuk, akun, ada, pend...","[masuk, akun, pendaftaran, akun, debit, klik, ..."
5,Ini Ono,"Semua menu menampilkan ""Permintaan anda belum ...",1,4.3.0,2023-01-06 12:47:03,-1.0,-1.0,-1.0,Negatif,Semua menu menampilkan Permintaan anda belum ...,semua menu menampilkan permintaan anda belum ...,"[semua, menu, menampilkan, permintaan, anda, b...","[menu, menampilkan, permintaan, akses, gak, be..."
6,Rismawati _S,"Aplikasi terlalu ribett, bukannya mempermudah ...",1,4.3.0,2023-01-27 12:04:18,-1.0,-1.0,-1.0,Negatif,Aplikasi terlalu ribett bukannya mempermudah ...,aplikasi terlalu ribett bukannya mempermudah ...,"[aplikasi, terlalu, ribett, bukannya, mempermu...","[aplikasi, ribett, mempermudah, mempersulit, m..."
7,Puspitasari Ifan,"setelah saya download dan login kembali, tp yg...",1,4.3.0,2023-01-05 04:05:51,-1.0,-1.0,-1.0,Negatif,setelah saya download dan login kembali tp yg...,setelah saya download dan login kembali tp yg...,"[setelah, saya, download, dan, login, kembali,...","[download, login, tp, yg, muncul, pin, utk, ve..."
8,Adam Sulaeman,"Setelah update aplikasinya, data di fitur Pend...",1,4.3.0,2023-01-01 09:17:00,-1.0,-1.0,-1.0,Negatif,Setelah update aplikasinya data di fitur Pend...,setelah update aplikasinya data di fitur pend...,"[setelah, update, aplikasinya, data, di, fitur...","[update, aplikasinya, data, fitur, pendaftaran..."
9,Hajriah Nur,"Formasinya sudah bagus banyak menu, tapi pas s...",1,4.3.0,2023-01-01 13:26:00,-1.0,-1.0,-1.0,Negatif,Formasinya sudah bagus banyak menu tapi pas s...,formasinya sudah bagus banyak menu tapi pas s...,"[formasinya, sudah, bagus, banyak, menu, tapi,...","[formasinya, bagus, menu, pas, merubah, data, ..."


In [12]:
# normalisasi
def normalisasi(tweet):
    kamus_slangword = eval(open("combined_slang_words.txt").read()) # Membuka dictionary slangword
    pattern = re.compile(r'\b( ' + '|'.join (kamus_slangword.keys())+r')\b') # Search pola kata (contoh kpn -> kapan)
    content = []
    for kata in tweet:
        filteredSlang = pattern.sub(lambda x: kamus_slangword[x.group()],kata) # Replace slangword berdasarkan pola review yg telah ditentukan
        content.append(filteredSlang.lower())
    tweet = content
    return tweet

df['Normalization'] = df['Stopword_Removal'].apply(lambda x: normalisasi(x))
df.head(10)

,userName,content,score,reviewCreatedVersion,at,Sentimen (Aurel),Sentimen (Ade),Sentimen (Eky),Sentimen Akhir,TEXT,case_folding,Tokenization,Stopword_Removal,Normalization
0,Made Darmawan,Saya kasi bintang 1 dulu ya. Tolong diperbaiki...,1,4.3.0,2023-01-22 12:04:08,-1.0,-1.0,-1.0,Negatif,Saya kasi bintang dulu ya Tolong diperbaiki...,saya kasi bintang dulu ya tolong diperbaiki...,"[saya, kasi, bintang, dulu, ya, tolong, diperb...","[kasi, bintang, ya, tolong, diperbaiki, kemamp...","[kasi, bintang, iya, tolong, diperbaiki, kemam..."
1,windi tripani,"Apknya memudahkan dalam perubahan data, ngecek...",5,4.3.0,2023-01-27 06:56:51,1.0,1.0,1.0,Positif,Apknya memudahkan dalam perubahan data ngecek...,apknya memudahkan dalam perubahan data ngecek...,"[apknya, memudahkan, dalam, perubahan, data, n...","[apknya, memudahkan, perubahan, data, ngecek, ...","[apknya, memudahkan, perubahan, data, ngecek, ..."
2,PURLAMBANG,Saya mau merubah data KIS saya di aplikasi jkn...,1,4.3.0,2023-01-27 09:16:17,-1.0,-1.0,-1.0,Negatif,Saya mau merubah data KIS saya di aplikasi jkn...,saya mau merubah data kis saya di aplikasi jkn...,"[saya, mau, merubah, data, kis, saya, di, apli...","[merubah, data, kis, aplikasi, jkn, layar, per...","[merubah, data, kis, aplikasi, jkn, layar, per..."
3,Nanang WR,"Faskes tidak bisa di search, dan yang ditampil...",1,4.3.0,2022-12-28 01:40:56,-1.0,-1.0,-1.0,Negatif,Faskes tidak bisa di search dan yang ditampil...,faskes tidak bisa di search dan yang ditampil...,"[faskes, tidak, bisa, di, search, dan, yang, d...","[faskes, search, ditampilkan, sesuai, gps, lha...","[faskes, search, ditampilkan, sesuai, gps, lha..."
4,Oktriyani Putri,Kenapa ketika ingin masuk akun ada pendaftaran...,1,4.3.0,2023-01-10 02:22:07,-1.0,-1.0,-1.0,Negatif,Kenapa ketika ingin masuk akun ada pendaftaran...,kenapa ketika ingin masuk akun ada pendaftaran...,"[kenapa, ketika, ingin, masuk, akun, ada, pend...","[masuk, akun, pendaftaran, akun, debit, klik, ...","[masuk, akun, pendaftaran, akun, debit, klik, ..."
5,Ini Ono,"Semua menu menampilkan ""Permintaan anda belum ...",1,4.3.0,2023-01-06 12:47:03,-1.0,-1.0,-1.0,Negatif,Semua menu menampilkan Permintaan anda belum ...,semua menu menampilkan permintaan anda belum ...,"[semua, menu, menampilkan, permintaan, anda, b...","[menu, menampilkan, permintaan, akses, gak, be...","[menu, menampilkan, permintaan, akses, gak, be..."
6,Rismawati _S,"Aplikasi terlalu ribett, bukannya mempermudah ...",1,4.3.0,2023-01-27 12:04:18,-1.0,-1.0,-1.0,Negatif,Aplikasi terlalu ribett bukannya mempermudah ...,aplikasi terlalu ribett bukannya mempermudah ...,"[aplikasi, terlalu, ribett, bukannya, mempermu...","[aplikasi, ribett, mempermudah, mempersulit, m...","[aplikasi, ribett, mempermudah, mempersulit, m..."
7,Puspitasari Ifan,"setelah saya download dan login kembali, tp yg...",1,4.3.0,2023-01-05 04:05:51,-1.0,-1.0,-1.0,Negatif,setelah saya download dan login kembali tp yg...,setelah saya download dan login kembali tp yg...,"[setelah, saya, download, dan, login, kembali,...","[download, login, tp, yg, muncul, pin, utk, ve...","[download, login, tapi, yang, muncul, pin, unt..."
8,Adam Sulaeman,"Setelah update aplikasinya, data di fitur Pend...",1,4.3.0,2023-01-01 09:17:00,-1.0,-1.0,-1.0,Negatif,Setelah update aplikasinya data di fitur Pend...,setelah update aplikasinya data di fitur pend...,"[setelah, update, aplikasinya, data, di, fitur...","[update, aplikasinya, data, fitur, pendaftaran...","[update, aplikasinya, data, fitur, pendaftaran..."
9,Hajriah Nur,"Formasinya sudah bagus banyak menu, tapi pas s...",1,4.3.0,2023-01-01 13:26:00,-1.0,-1.0,-1.0,Negatif,Formasinya sudah bagus banyak menu tapi pas s...,formasinya sudah bagus banyak menu tapi pas s...,"[formasinya, sudah, bagus, banyak, menu, tapi,...","[formasinya, bagus, menu, pas, merubah, data, ...","[formasinya, bagus, menu, pas, merubah, data, ..."


In [14]:
# stemming
factory = StemmerFactory()
stemming = factory.create_stemmer()

# Modify the stem_list function to take a row as input and apply stemming to each word in the 'Normalization' column
def stem_list(row):
    # Access the 'Normalization' column for the current row
    words = row['Normalization']
    # Apply stemming to each word in the list and return a list of stemmed words
    stemmed_words = [stemming.stem(word) for word in words]
    return stemmed_words

# Apply the modified function to the DataFrame along axis 1 (row-wise)
df['Stemming'] = df.apply(stem_list, axis=1)
df.head(30)

,userName,content,score,reviewCreatedVersion,at,Sentimen (Aurel),Sentimen (Ade),Sentimen (Eky),Sentimen Akhir,TEXT,case_folding,Tokenization,Stopword_Removal,Normalization,Stemming
0,Made Darmawan,Saya kasi bintang 1 dulu ya. Tolong diperbaiki...,1,4.3.0,2023-01-22 12:04:08,-1.0,-1.0,-1.0,Negatif,Saya kasi bintang dulu ya Tolong diperbaiki...,saya kasi bintang dulu ya tolong diperbaiki...,"[saya, kasi, bintang, dulu, ya, tolong, diperb...","[kasi, bintang, ya, tolong, diperbaiki, kemamp...","[kasi, bintang, iya, tolong, diperbaiki, kemam...","[kasi, bintang, iya, tolong, baik, mampu, scro..."
1,windi tripani,"Apknya memudahkan dalam perubahan data, ngecek...",5,4.3.0,2023-01-27 06:56:51,1.0,1.0,1.0,Positif,Apknya memudahkan dalam perubahan data ngecek...,apknya memudahkan dalam perubahan data ngecek...,"[apknya, memudahkan, dalam, perubahan, data, n...","[apknya, memudahkan, perubahan, data, ngecek, ...","[apknya, memudahkan, perubahan, data, ngecek, ...","[apknya, mudah, ubah, data, ngecek, tagih, riw..."
2,PURLAMBANG,Saya mau merubah data KIS saya di aplikasi jkn...,1,4.3.0,2023-01-27 09:16:17,-1.0,-1.0,-1.0,Negatif,Saya mau merubah data KIS saya di aplikasi jkn...,saya mau merubah data kis saya di aplikasi jkn...,"[saya, mau, merubah, data, kis, saya, di, apli...","[merubah, data, kis, aplikasi, jkn, layar, per...","[merubah, data, kis, aplikasi, jkn, layar, per...","[rubah, data, kis, aplikasi, jkn, layar, ubah,..."
3,Nanang WR,"Faskes tidak bisa di search, dan yang ditampil...",1,4.3.0,2022-12-28 01:40:56,-1.0,-1.0,-1.0,Negatif,Faskes tidak bisa di search dan yang ditampil...,faskes tidak bisa di search dan yang ditampil...,"[faskes, tidak, bisa, di, search, dan, yang, d...","[faskes, search, ditampilkan, sesuai, gps, lha...","[faskes, search, ditampilkan, sesuai, gps, lha...","[faskes, search, tampil, sesuai, gps, lha, kan..."
4,Oktriyani Putri,Kenapa ketika ingin masuk akun ada pendaftaran...,1,4.3.0,2023-01-10 02:22:07,-1.0,-1.0,-1.0,Negatif,Kenapa ketika ingin masuk akun ada pendaftaran...,kenapa ketika ingin masuk akun ada pendaftaran...,"[kenapa, ketika, ingin, masuk, akun, ada, pend...","[masuk, akun, pendaftaran, akun, debit, klik, ...","[masuk, akun, pendaftaran, akun, debit, klik, ...","[masuk, akun, daftar, akun, debit, klik, sedia..."
5,Ini Ono,"Semua menu menampilkan ""Permintaan anda belum ...",1,4.3.0,2023-01-06 12:47:03,-1.0,-1.0,-1.0,Negatif,Semua menu menampilkan Permintaan anda belum ...,semua menu menampilkan permintaan anda belum ...,"[semua, menu, menampilkan, permintaan, anda, b...","[menu, menampilkan, permintaan, akses, gak, be...","[menu, menampilkan, permintaan, akses, gak, be...","[menu, tampil, minta, akses, gak, fungsi, bang..."
6,Rismawati _S,"Aplikasi terlalu ribett, bukannya mempermudah ...",1,4.3.0,2023-01-27 12:04:18,-1.0,-1.0,-1.0,Negatif,Aplikasi terlalu ribett bukannya mempermudah ...,aplikasi terlalu ribett bukannya mempermudah ...,"[aplikasi, terlalu, ribett, bukannya, mempermu...","[aplikasi, ribett, mempermudah, mempersulit, m...","[aplikasi, ribett, mempermudah, mempersulit, m...","[aplikasi, ribett, mudah, sulit, akses, daftar..."
7,Puspitasari Ifan,"setelah saya download dan login kembali, tp yg...",1,4.3.0,2023-01-05 04:05:51,-1.0,-1.0,-1.0,Negatif,setelah saya download dan login kembali tp yg...,setelah saya download dan login kembali tp yg...,"[setelah, saya, download, dan, login, kembali,...","[download, login, tp, yg, muncul, pin, utk, ve...","[download, login, tapi, yang, muncul, pin, unt...","[download, login, tapi, yang, muncul, pin, unt..."
8,Adam Sulaeman,"Setelah update aplikasinya, data di fitur Pend...",1,4.3.0,2023-01-01 09:17:00,-1.0,-1.0,-1.0,Negatif,Setelah update aplikasinya data di fitur Pend...,setelah update aplikasinya data di fitur pend...,"[setelah, update, aplikasinya, data, di, fitur...","[update, aplikasinya, data, fitur, pendaftaran...","[update, aplikasinya, data, fitur, pendaftaran...","[update, aplikasi, data, fitur, daftar, layan,..."
9,Hajriah N

In [15]:
# Gabungkan token hasil normalisasi menjadi string kembali (optional, tergantung input model)
df['final_text'] = df['Normalization'].apply(lambda x: ' '.join(x))

# --- SPLIT DATA ---

# Misalnya kolom label bernama 'label' (ganti jika nama kolom berbeda)
X = df['final_text']
y = df['Sentimen Akhir']  # ganti dengan nama kolom label di file Excel kamu

# hapus nilai NaN
df_cleaned = df.dropna(subset=['Sentimen Akhir']).copy()
X_cleaned = df_cleaned['final_text']
y_cleaned = df_cleaned['Sentimen Akhir']

# Split: 80% train, 20% test dengan stratifikasi agar distribusi label seimbang
X_train, X_test, y_train, y_test = train_test_split(
    X_cleaned, y_cleaned, test_size=0.2, stratify=y_cleaned, random_state=42
)

print("Jumlah data latih:", len(X_train))
print("Jumlah data uji:", len(X_test))

Jumlah data latih: 8601
Jumlah data uji: 2151


In [16]:
# --- Crucial Correction: Encode target labels BEFORE splitting ---
df_cleaned = df.dropna(subset=['Sentimen Akhir']).copy()

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'Sentimen Akhir' column to numerical labels
# This is the most important part for your error!
y_encoded = label_encoder.fit_transform(df_cleaned['Sentimen Akhir'])

# Now define X_cleaned and y_cleaned_encoded
X_cleaned = df_cleaned['final_text']
y_cleaned_encoded = y_encoded # Use the encoded labels here!


# --- SPLIT DATA ---
X_train, X_test, y_train, y_test = train_test_split(
    X_cleaned, y_cleaned_encoded, test_size=0.2, stratify=y_cleaned_encoded, random_state=42
)

print("Jumlah data latih:", len(X_train))
print("Jumlah data uji:", len(X_test))

Jumlah data latih: 8601
Jumlah data uji: 2151


In [17]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'Sentimen Akhir' column
# Ensure you fit on the cleaned column before splitting
y_encoded = label_encoder.fit_transform(df_cleaned['Sentimen Akhir'])
# You can check the mapping:
print("Original labels:", label_encoder.classes_)
print("Encoded labels:", np.unique(y_encoded))


# --- SPLIT DATA (updated to use encoded labels) ---

X_cleaned = df_cleaned['final_text']
# Use the encoded labels here
y_cleaned_encoded = y_encoded

# Split: 80% train, 20% test with stratifikasi agar distribusi label seimbang
X_train, X_test, y_train, y_test = train_test_split(X_cleaned, y_cleaned_encoded, test_size=0.2, stratify=y_cleaned_encoded, random_state=42)

print("Jumlah data latih:", len(X_train))
print("Jumlah data uji:", len(X_test))

Original labels: ['Negatif' 'Netral' 'Positif']
Encoded labels: [0 1 2]
Jumlah data latih: 8601
Jumlah data uji: 2151


In [18]:
# Feature Extraction: TF-IDF
tfidf = TfidfVectorizer(max_features=1000)  # Batasi fitur untuk efisiensi
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

In [19]:
# SMOTE version
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_vec, y_train)

# **SVM**

In [ ]:
svm = SVC(kernel='rbf', C=1, gamma=1)

start = time.time()
svm.fit(X_train_vec, y_train)
pred = svm.predict(X_test_vec)
elapsed = time.time() - start

print("🔹 SVM")
acc = accuracy_score(y_test, pred)
print("Accuracy: {:.4f}".format(acc))
print(classification_report(y_test, pred))
print("Elapsed time (s): {:.4f}".format(elapsed))

🔹 SVM
              precision    recall  f1-score   support

           0       0.91      0.93      0.92       481
           1       0.75      0.07      0.13        42
           2       0.97      0.98      0.98      1628

    accuracy                           0.95      2151
   macro avg       0.88      0.66      0.67      2151
weighted avg       0.95      0.95      0.95      2151

Accuracy: 0.9535
              precision    recall  f1-score   support

           0       0.91      0.93      0.92       481
           1       0.75      0.07      0.13        42
           2       0.97      0.98      0.98      1628

    accuracy                           0.95      2151
   macro avg       0.88      0.66      0.67      2151
weighted avg       0.95      0.95      0.95      2151

Elapsed time (s): 2.0004


SVM + SMOTE

In [ ]:
svm_smote = SVC(kernel='rbf', C=1, gamma=1)

start = time.time()
svm_smote.fit(X_train_smote, y_train_smote)
pred = svm_smote.predict(X_test_vec)
elapsed = time.time() - start

print("🔹 SVM + SMOTE")
print(classification_report(y_test, pred))
print(f"Accuracy: {accuracy_score(y_test, pred):.4f}")
print(f"Elapsed time (s): {elapsed:.4f}")


🔹 SVM + SMOTE
              precision    recall  f1-score   support

           0       0.90      0.93      0.91       481
           1       0.06      0.19      0.09        42
           2       0.98      0.91      0.94      1628

    accuracy                           0.90      2151
   macro avg       0.64      0.68      0.65      2151
weighted avg       0.94      0.90      0.92      2151

Accuracy: 0.9005
Elapsed time (s): 10.1817


SVM + RS

In [ ]:
param_dist_svm_rs = {
    'C': [1],
    'gamma': [1],
    'kernel': ['rbf']
}

random_search = RandomizedSearchCV(SVC(), param_distributions=param_dist_svm_rs, n_iter=10, cv=5, random_state=42, n_jobs=-1)
start = time.time()
random_search.fit(X_train_vec, y_train)
pred = random_search.predict(X_test_vec)
elapsed = time.time() - start

print("🔹 SVM + RS")
acc = accuracy_score(y_test, pred)
print("Accuracy: {:.4f}".format(acc))
print(classification_report(y_test, pred))
print("Elapsed Time: {:.2f} s".format(elapsed))


🔹 SVM + RS
Accuracy: 0.9535
              precision    recall  f1-score   support

           0       0.91      0.93      0.92       481
           1       0.75      0.07      0.13        42
           2       0.97      0.98      0.98      1628

    accuracy                           0.95      2151
   macro avg       0.88      0.66      0.67      2151
weighted avg       0.95      0.95      0.95      2151

Elapsed Time: 9.70 s


SVM + RS + SMOTE

In [ ]:
random_search_smote = RandomizedSearchCV(SVC(), param_distributions=param_dist_svm_rs, n_iter=10, cv=5, random_state=42)
start = time.time()
random_search_smote.fit(X_train_smote, y_train_smote)
pred = random_search_smote.predict(X_test_vec)
elapsed = time.time() - start
print("🔹 SVM + RS + SMOTE")
print(classification_report(y_test, pred))
print(f"Accuracy: {accuracy_score(y_test, pred):.4f}")
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 SVM + RS + SMOTE
              precision    recall  f1-score   support

           0       0.90      0.93      0.91       481
           1       0.06      0.19      0.09        42
           2       0.98      0.91      0.94      1628

    accuracy                           0.90      2151
   macro avg       0.64      0.68      0.65      2151
weighted avg       0.94      0.90      0.92      2151

Accuracy: 0.9005
Elapsed Time: 48.89 s


SVM + BO

In [ ]:
bayes = BayesSearchCV(
    SVC(),
    search_spaces={
        'C': [1],
        'gamma': [1],
        'kernel': ['rbf']
    },
    n_iter=10,
    cv=5,
    random_state=42
)

start = time.time()
bayes.fit(X_train_vec, y_train)
pred = bayes.predict(X_test_vec)
elapsed = time.time() - start

print("🔹 SVM + BO")
acc = accuracy_score(y_test, pred)
print("Accuracy: {:.4f}".format(acc))
print(classification_report(y_test, pred))
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 SVM + BO
Accuracy: 0.9535
              precision    recall  f1-score   support

           0       0.91      0.93      0.92       481
           1       0.75      0.07      0.13        42
           2       0.97      0.98      0.98      1628

    accuracy                           0.95      2151
   macro avg       0.88      0.66      0.67      2151
weighted avg       0.95      0.95      0.95      2151

Elapsed Time: 72.67 s


SVM + BO + SMOTE

In [ ]:
bayes_smote = BayesSearchCV(
    SVC(),
    search_spaces={
        'C': (1e-2, 1e2, 'log-uniform'),
        'gamma': (1e-3, 1e1, 'log-uniform'),
        'kernel': ['rbf']
    },
    n_iter=10,
    cv=5,
    random_state=42
)

start = time.time()
bayes_smote.fit(X_train_smote, y_train_smote)
pred = bayes_smote.predict(X_test_vec)
elapsed = time.time() - start

print("🔹 SVM + BO + SMOTE")
acc = accuracy_score(y_test, pred)
print("Accuracy: {:.4f}".format(acc))
print(classification_report(y_test, pred))
print("Best Params:",bayes_smote.best_params_)
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 SVM + BO + SMOTE
Accuracy: 0.8824
              precision    recall  f1-score   support

           0       0.89      0.85      0.87       481
           1       0.06      0.17      0.09        42
           2       0.94      0.91      0.93      1628

    accuracy                           0.88      2151
   macro avg       0.63      0.64      0.63      2151
weighted avg       0.91      0.88      0.90      2151

Best Params: OrderedDict([('C', 22.364202820542708), ('gamma', 3.4139810765573975), ('kernel', 'rbf')])
Elapsed Time: 593.19 s


SVM + PSO

In [ ]:
def objective(trial):
    C = trial.suggest_loguniform('C', 1e-2, 1e2)
    gamma = trial.suggest_loguniform('gamma', 1e-3, 1e1)
    model = SVC(kernel='rbf', C=C, gamma=gamma)
    score = cross_val_score(model, X_train_vec, y_train, cv=5, scoring='accuracy').mean()
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

best_params = study.best_params
model = SVC(kernel='rbf', **best_params)

start = time.time()
model.fit(X_train_vec, y_train)
pred = model.predict(X_test_vec)
elapsed = time.time() - start

acc = accuracy_score(y_test, pred)

print("🔹 SVM + PSO (Optuna)")
print("Accuracy: {:.4f}".format(acc))
print(classification_report(y_test, pred))
print("Elapsed Time: {:.2f} s".format(elapsed))
print("Elapsed Time: {:.2f} s".format(elapsed))

[I 2025-06-27 08:54:23,931] A new study created in memory with name: no-name-ecf8d935-f23a-455e-9a34-e84d97e3bae0
[I 2025-06-27 08:54:28,703] Trial 0 finished with value: 0.9461680607542938 and parameters: {'C': 6.55212595038684, 'gamma': 0.1935442632567802}. Best is trial 0 with value: 0.9461680607542938.
[I 2025-06-27 08:54:38,119] Trial 1 finished with value: 0.7568887747794009 and parameters: {'C': 0.01313818596600234, 'gamma': 0.006618247528604469}. Best is trial 0 with value: 0.9461680607542938.
[I 2025-06-27 08:54:42,307] Trial 2 finished with value: 0.9454703187708606 and parameters: {'C': 58.771186354646126, 'gamma': 0.010003808464411212}. Best is trial 0 with value: 0.9461680607542938.
[I 2025-06-27 08:54:46,484] Trial 3 finished with value: 0.9452378957609826 and parameters: {'C': 42.418700999051914, 'gamma': 0.016723048435084983}. Best is trial 0 with value: 0.9461680607542938.
[I 2025-06-27 08:55:02,342] Trial 4 finished with value: 0.7568887747794009 and parameters: {'C':

🔹 SVM + PSO (Optuna)
Accuracy: 0.9470
              precision    recall  f1-score   support

           0       0.90      0.91      0.90       481
           1       0.42      0.12      0.19        42
           2       0.97      0.98      0.97      1628

    accuracy                           0.95      2151
   macro avg       0.76      0.67      0.69      2151
weighted avg       0.94      0.95      0.94      2151

Elapsed Time: 1.67 s
Elapsed Time: 1.67 s


SVM + PSO + SMOTE

In [ ]:
# Objective Function
def objective(trial):
    C = trial.suggest_loguniform('C', 1e-2, 1e2)
    gamma = trial.suggest_loguniform('gamma', 1e-3, 1e1)
    model = SVC(kernel='rbf', C=C, gamma=gamma)
    score = cross_val_score(model, X_train_smote, y_train_smote, cv=5, scoring='accuracy').mean()
    return score

# Run Optuna Optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Build Final Model
best_params = study.best_params
model = SVC(kernel='rbf', **best_params)

start = time.time()
model.fit(X_train_vec, y_train)
pred = model.predict(X_test_vec)
elapsed = time.time() - start

acc = accuracy_score(y_test, pred)

# Output Results
print("🔹 SVM + PSO (Optuna) + SMOTE")
print("Accuracy: {:.4f}".format(acc))
print(classification_report(y_test, pred))
print("Elapsed Time: {:.2f} s".format(elapsed))

[I 2025-06-27 08:55:49,444] A new study created in memory with name: no-name-93b6416c-63b1-4365-acaa-bd873e8627b0
[I 2025-06-27 08:56:16,197] Trial 0 finished with value: 0.931131592421915 and parameters: {'C': 69.08136213505458, 'gamma': 0.0964837626098978}. Best is trial 0 with value: 0.931131592421915.
[I 2025-06-27 08:56:53,585] Trial 1 finished with value: 0.9008192524321557 and parameters: {'C': 10.761754144744442, 'gamma': 0.04776434844435353}. Best is trial 0 with value: 0.931131592421915.
[I 2025-06-27 08:58:13,467] Trial 2 finished with value: 0.8772145417306708 and parameters: {'C': 0.2294684724300371, 'gamma': 3.3396588004248193}. Best is trial 0 with value: 0.931131592421915.
[I 2025-06-27 09:00:06,013] Trial 3 finished with value: 0.758115719406042 and parameters: {'C': 0.01826951185977066, 'gamma': 1.3483353877898567}. Best is trial 0 with value: 0.931131592421915.
[I 2025-06-27 09:02:32,507] Trial 4 finished with value: 0.6336917562724015 and parameters: {'C': 0.4837383

🔹 SVM + PSO (Optuna) + SMOTE
Accuracy: 0.9479
              precision    recall  f1-score   support

           0       0.90      0.91      0.91       481
           1       0.40      0.05      0.09        42
           2       0.96      0.98      0.97      1628

    accuracy                           0.95      2151
   macro avg       0.76      0.65      0.66      2151
weighted avg       0.94      0.95      0.94      2151

Elapsed Time: 3.86 s


# **KNN**

KNN

In [ ]:
knn_model = KNeighborsClassifier()
start = time.time()
knn_model.fit(X_train_vec, y_train)
knn_pred = knn_model.predict(X_test_vec)
elapsed = time.time() - start

print("🔹 KNN")
acc = accuracy_score(y_test, knn_pred)
print("Accuracy: {:.4f}".format(acc))
print(classification_report(y_test, knn_pred))
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 KNN
Accuracy: 0.8126
              precision    recall  f1-score   support

           0       0.88      0.27      0.42       481
           1       0.00      0.00      0.00        42
           2       0.81      0.99      0.89      1628

    accuracy                           0.81      2151
   macro avg       0.56      0.42      0.44      2151
weighted avg       0.81      0.81      0.77      2151

Elapsed Time: 0.34 s


KNN + SMOTE

In [ ]:
knn_smote_model = KNeighborsClassifier()
start = time.time()
knn_smote_model.fit(X_train_smote, y_train_smote)
knn_smote_pred = knn_smote_model.predict(X_test_vec)
elapsed = time.time() - start

print("🔹 KNN + SMOTE")
acc = accuracy_score(y_test, knn_pred)
print("Accuracy: {:.4f}".format(acc))
print(classification_report(y_test, knn_smote_pred))
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 KNN + SMOTE
Accuracy: 0.8126
              precision    recall  f1-score   support

           0       0.73      0.69      0.71       481
           1       0.06      0.52      0.10        42
           2       0.99      0.79      0.88      1628

    accuracy                           0.77      2151
   macro avg       0.59      0.67      0.56      2151
weighted avg       0.91      0.77      0.83      2151

Elapsed Time: 0.77 s


KNN + RS

In [ ]:
knn_param_dist = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

rs_knn = RandomizedSearchCV(KNeighborsClassifier(), param_distributions=knn_param_dist,
                            n_iter=10, cv=5, random_state=42, n_jobs=-1)
start = time.time()
rs_knn.fit(X_train_vec, y_train)
rs_knn_pred = rs_knn.predict(X_test_vec)
elapsed = time.time() - start

print("🔹 RS (with best parameters)")
acc = accuracy_score(y_test, rs_knn_pred)
print("Accuracy: {:.4f}".format(acc))
print(classification_report(y_test, rs_knn_pred))
print(f"Best parameters for RS: {rs_knn.best_params_}")
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 RS (with best parameters)
Accuracy: 0.8275
              precision    recall  f1-score   support

           0       0.86      0.36      0.50       481
           1       0.33      0.10      0.15        42
           2       0.83      0.99      0.90      1628

    accuracy                           0.83      2151
   macro avg       0.67      0.48      0.52      2151
weighted avg       0.83      0.83      0.80      2151

Best parameters for RS: {'weights': 'distance', 'n_neighbors': 3, 'metric': 'minkowski'}
Elapsed Time: 15.24 s


KNN + RS + SMOTE

In [ ]:
rs_knn_smote = RandomizedSearchCV(KNeighborsClassifier(), param_distributions=knn_param_dist,
                                  n_iter=10, cv=5, random_state=42, n_jobs=-1)
start = time.time()
rs_knn_smote.fit(X_train_smote, y_train_smote)
rs_knn_smote_pred = rs_knn_smote.predict(X_test_vec)
elapsed = time.time() - start

print("🔹 RS + SMOTE (with best parameters)")
acc = accuracy_score(y_test, rs_knn_smote_pred)
print("Accuracy: {:.4f}".format(acc))
print(classification_report(y_test, rs_knn_smote_pred))
print(f"Best parameters for RS + SMOTE: {rs_knn_smote.best_params_}")
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 RS + SMOTE (with best parameters)
Accuracy: 0.8020
              precision    recall  f1-score   support

           0       0.77      0.75      0.76       481
           1       0.06      0.45      0.11        42
           2       0.98      0.83      0.90      1628

    accuracy                           0.80      2151
   macro avg       0.60      0.68      0.59      2151
weighted avg       0.92      0.80      0.85      2151

Best parameters for RS + SMOTE: {'weights': 'distance', 'n_neighbors': 3, 'metric': 'minkowski'}
Elapsed Time: 74.49 s


KNN + BO

In [ ]:
bayes_knn_search_spaces = {
    'n_neighbors': Integer(3, 15),
    'weights': Categorical(['uniform', 'distance']),
    'metric': Categorical(['euclidean', 'manhattan', 'minkowski'])
}

bo_knn = BayesSearchCV(
    KNeighborsClassifier(),
    search_spaces=bayes_knn_search_spaces,
    n_iter=10,
    cv=5,
    random_state=42,
    n_jobs=-1
)
start = time.time()
bo_knn.fit(X_train_vec, y_train)
bo_knn_pred = bo_knn.predict(X_test_vec)
elapsed = time.time() - start

print("🔹 BO (with best parameters)")
acc = accuracy_score(y_test, bo_knn_pred)
print("Accuracy: {:.4f}".format(acc))
print(classification_report(y_test, bo_knn_pred))
print(f"Best parameters for BO: {bo_knn.best_params_}")
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 BO (with best parameters)
Accuracy: 0.8131
              precision    recall  f1-score   support

           0       0.88      0.27      0.41       481
           1       0.38      0.07      0.12        42
           2       0.81      0.99      0.89      1628

    accuracy                           0.81      2151
   macro avg       0.69      0.44      0.48      2151
weighted avg       0.82      0.81      0.77      2151

Best parameters for BO: OrderedDict([('metric', 'minkowski'), ('n_neighbors', 5), ('weights', 'distance')])
Elapsed Time: 17.09 s


KNN + BO + SMOTE

In [ ]:
bo_knn_smote = BayesSearchCV(
    KNeighborsClassifier(),
    search_spaces=bayes_knn_search_spaces,
    n_iter=10,
    cv=5,
    random_state=42,
    n_jobs=-1
)
start = time.time()
bo_knn_smote.fit(X_train_smote, y_train_smote)
bo_knn_smote_pred = bo_knn_smote.predict(X_test_vec)
elapsed = time.time() - start

print("🔹 BO + SMOTE (with best parameters)")
acc = accuracy_score(y_test, bo_knn_smote_pred)
print("Accuracy: {:.4f}".format(acc))
print(classification_report(y_test, bo_knn_smote_pred))
print(f"Best parameters for BO + SMOTE: {bo_knn_smote.best_params_}")
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 BO + SMOTE (with best parameters)
Accuracy: 0.7768
              precision    recall  f1-score   support

           0       0.75      0.69      0.72       481
           1       0.06      0.50      0.10        42
           2       0.99      0.81      0.89      1628

    accuracy                           0.78      2151
   macro avg       0.60      0.67      0.57      2151
weighted avg       0.92      0.78      0.84      2151

Best parameters for BO + SMOTE: OrderedDict([('metric', 'minkowski'), ('n_neighbors', 5), ('weights', 'distance')])
Elapsed Time: 92.55 s


KNN + PSO

In [ ]:
def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 15)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 'minkowski'])
    model = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, metric=metric)
    score = cross_val_score(model, X_train_vec, y_train, cv=5, scoring='accuracy', n_jobs=-1).mean()
    return score

start = time.time()
study_knn = optuna.create_study(direction='maximize')
study_knn.optimize(objective_knn, n_trials=10)
best_params_knn = study_knn.best_params
optuna_knn_model = KNeighborsClassifier(**best_params_knn)
optuna_knn_model.fit(X_train_vec, y_train)
optuna_knn_pred = optuna_knn_model.predict(X_test_vec)
elapsed = time.time() - start

print("🔹 PSO (Optuna) with best parameters")
acc = accuracy_score(y_test, optuna_knn_pred)
print("Accuracy: {:.4f}".format(acc))
print(classification_report(y_test, optuna_knn_pred))
print(f"Best parameters for PSO (Optuna): {best_params_knn}")
print("Elapsed Time: {:.2f} s".format(elapsed))

[I 2025-06-27 09:13:02,682] A new study created in memory with name: no-name-e9e4baae-9a04-4fd7-a8ea-2b059e2c82fc
[I 2025-06-27 09:13:03,590] Trial 0 finished with value: 0.7739797035255328 and parameters: {'n_neighbors': 15, 'weights': 'uniform', 'metric': 'minkowski'}. Best is trial 0 with value: 0.7739797035255328.
[I 2025-06-27 09:13:05,831] Trial 1 finished with value: 0.77060747537262 and parameters: {'n_neighbors': 11, 'weights': 'uniform', 'metric': 'manhattan'}. Best is trial 0 with value: 0.7739797035255328.
[I 2025-06-27 09:13:07,250] Trial 2 finished with value: 0.7790954420766726 and parameters: {'n_neighbors': 14, 'weights': 'distance', 'metric': 'minkowski'}. Best is trial 2 with value: 0.7790954420766726.
[I 2025-06-27 09:13:08,435] Trial 3 finished with value: 0.7917682391254409 and parameters: {'n_neighbors': 8, 'weights': 'uniform', 'metric': 'minkowski'}. Best is trial 3 with value: 0.7917682391254409.
[I 2025-06-27 09:13:09,355] Trial 4 finished with value: 0.79444

🔹 PSO (Optuna) with best parameters
Accuracy: 0.8024
              precision    recall  f1-score   support

           0       0.89      0.22      0.35       481
           1       0.50      0.07      0.12        42
           2       0.80      0.99      0.89      1628

    accuracy                           0.80      2151
   macro avg       0.73      0.43      0.45      2151
weighted avg       0.81      0.80      0.75      2151

Best parameters for PSO (Optuna): {'n_neighbors': 7, 'weights': 'distance', 'metric': 'euclidean'}
Elapsed Time: 14.44 s


KNN + PSO + SMOTE

In [ ]:
def objective_knn_smote(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 15)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 'minkowski'])
    model = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, metric=metric)
    score = cross_val_score(model, X_train_smote, y_train_smote, cv=5, scoring='accuracy', n_jobs=-1).mean()
    return score

start = time.time()
study_knn_smote = optuna.create_study(direction='maximize')
study_knn_smote.optimize(objective_knn_smote, n_trials=10)
best_params_knn_smote = study_knn_smote.best_params
optuna_knn_smote_model = KNeighborsClassifier(**best_params_knn_smote)
optuna_knn_smote_model.fit(X_train_smote, y_train_smote)
optuna_knn_smote_pred = optuna_knn_smote_model.predict(X_test_vec)
elapsed = time.time() - start

print("🔹 PSO (Optuna) + SMOTE with best parameters")
acc = accuracy_score(y_test, optuna_knn_smote_pred)
print("Accuracy: {:.4f}".format(acc))
print(classification_report(y_test, optuna_knn_smote_pred))
print(f"Best parameters for PSO (Optuna) + SMOTE: {best_params_knn_smote}")
print("Elapsed Time: {:.2f} s".format(elapsed))

[I 2025-06-27 09:13:17,135] A new study created in memory with name: no-name-5729c1b7-0887-4a77-ae1e-5089cd065e9f
[I 2025-06-27 09:13:22,311] Trial 0 finished with value: 0.7963133640552995 and parameters: {'n_neighbors': 12, 'weights': 'uniform', 'metric': 'minkowski'}. Best is trial 0 with value: 0.7963133640552995.
[I 2025-06-27 09:13:37,946] Trial 1 finished with value: 0.8416282642089093 and parameters: {'n_neighbors': 3, 'weights': 'uniform', 'metric': 'manhattan'}. Best is trial 1 with value: 0.8416282642089093.
[I 2025-06-27 09:13:53,598] Trial 2 finished with value: 0.7567332309267792 and parameters: {'n_neighbors': 15, 'weights': 'distance', 'metric': 'manhattan'}. Best is trial 1 with value: 0.8416282642089093.
[I 2025-06-27 09:13:58,566] Trial 3 finished with value: 0.8513568868407578 and parameters: {'n_neighbors': 6, 'weights': 'uniform', 'metric': 'minkowski'}. Best is trial 3 with value: 0.8513568868407578.
[I 2025-06-27 09:14:03,501] Trial 4 finished with value: 0.8513

🔹 PSO (Optuna) + SMOTE with best parameters
Accuracy: 0.7615
              precision    recall  f1-score   support

           0       0.74      0.65      0.69       481
           1       0.06      0.57      0.11        42
           2       0.99      0.80      0.88      1628

    accuracy                           0.76      2151
   macro avg       0.60      0.67      0.56      2151
weighted avg       0.91      0.76      0.83      2151

Best parameters for PSO (Optuna) + SMOTE: {'n_neighbors': 7, 'weights': 'distance', 'metric': 'minkowski'}
Elapsed Time: 81.84 s


# **Random Forest**

Random Forest (RF)

In [ ]:
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    max_features='sqrt',
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
) #default value

start = time.time()

rf.fit(X_train_vec, y_train)
pred_rf = rf.predict(X_test_vec)

elapsed = time.time() - start

print("🔹 Random Forest")
print(classification_report(y_test, pred_rf))
acc = accuracy_score(y_test, pred_rf)
print("\nAccuracy: {:.4f}".format(acc))
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 Random Forest
              precision    recall  f1-score   support

           0       0.87      0.89      0.88       481
           1       0.43      0.07      0.12        42
           2       0.95      0.97      0.96      1628

    accuracy                           0.93      2151
   macro avg       0.75      0.64      0.65      2151
weighted avg       0.93      0.93      0.93      2151


Accuracy: 0.9340
Elapsed Time: 1.98 s


RF + SMOTE

In [ ]:
rf_smote = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    max_features='sqrt',
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
) #default value

start = time.time()

rf_smote.fit(X_train_smote, y_train_smote)
pred_rf_smote = rf_smote.predict(X_test_vec)

elapsed = time.time() - start

print("🔹 Random Forest + SMOTE")
print(classification_report(y_test, pred_rf_smote))
acc = accuracy_score(y_test, pred_rf_smote)
print("\nAccuracy: {:.4f}".format(acc))
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 Random Forest + SMOTE
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       481
           1       0.20      0.24      0.22        42
           2       0.97      0.96      0.96      1628

    accuracy                           0.93      2151
   macro avg       0.68      0.70      0.69      2151
weighted avg       0.93      0.93      0.93      2151


Accuracy: 0.9279
Elapsed Time: 5.07 s


RF + Random Search (RS)

In [ ]:
param_dist_rf = {
    'n_estimators': randint(100, 300),
    'max_depth': randint(10, 50),
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5)
}

In [ ]:
random_search_rf = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    param_distributions=param_dist_rf,
    n_iter=10,  # Jumlah kombinasi acak yang dicoba
    cv=5,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)

start = time.time()

random_search_rf.fit(X_train_vec, y_train)
pred_rf_rs = random_search_rf.predict(X_test_vec)

elapsed = time.time() - start

print("🔹 Random Search (RF + RS)")
print(classification_report(y_test, pred_rf_rs))
acc = accuracy_score(y_test, pred_rf_rs)
print("\nBest Parameters:", random_search_rf.best_params_)
print("\nAccuracy: {:.4f}".format(acc))
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 Random Search (RF + RS)
              precision    recall  f1-score   support

           0       0.90      0.78      0.84       481
           1       0.00      0.00      0.00        42
           2       0.92      0.98      0.95      1628

    accuracy                           0.92      2151
   macro avg       0.61      0.59      0.59      2151
weighted avg       0.90      0.92      0.90      2151


Best Parameters: {'max_depth': 36, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 287}

Accuracy: 0.9154
Elapsed Time: 33.19 s


RF + RS + SMOTE

In [ ]:
# Random Search CV dengan data hasil SMOTE
random_search_rf_smote = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    param_distributions=param_dist_rf,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)

start = time.time()

random_search_rf_smote.fit(X_train_smote, y_train_smote)
pred_rf_rs_smote = random_search_rf_smote.predict(X_test_vec)

elapsed = time.time() - start

print("🔹 Random Search (RF + RS + SMOTE)")
print(classification_report(y_test, pred_rf_rs_smote))
acc = accuracy_score(y_test, pred_rf_rs_smote)
print("\nBest Parameters:", random_search_rf_smote.best_params_)
print("\nAccuracy: {:.4f}".format(acc))
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 Random Search (RF + RS + SMOTE)
              precision    recall  f1-score   support

           0       0.89      0.83      0.86       481
           1       0.26      0.29      0.27        42
           2       0.95      0.97      0.96      1628

    accuracy                           0.92      2151
   macro avg       0.70      0.70      0.70      2151
weighted avg       0.93      0.92      0.92      2151


Best Parameters: {'max_depth': 48, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 8, 'n_estimators': 206}

Accuracy: 0.9247
Elapsed Time: 74.25 s


RF + Bayesian Optimization (BO)

In [ ]:
# Search space untuk Random Forest (BO)
search_spaces_rf = {
    'n_estimators': Integer(100, 300),
    'max_depth': Integer(10, 50),
    'max_features': Categorical(['sqrt', 'log2']),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 5)
}

In [ ]:
bayes_rf = BayesSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    search_spaces=search_spaces_rf,
    n_iter=10,  # jumlah iterasi pencarian
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)

start = time.time()

# Training dan evaluasi
bayes_rf.fit(X_train_vec, y_train)
pred_rf_bo = bayes_rf.predict(X_test_vec)

elapsed = time.time() - start

print("🔹 Bayesian Optimization (RF + BO)")
print(classification_report(y_test, pred_rf_bo))
acc = accuracy_score(y_test, pred_rf_bo)
print("\nBest Parameters:", bayes_rf.best_params_)
print("\nAccuracy: {:.4f}".format(acc))
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 Bayesian Optimization (RF + BO)
              precision    recall  f1-score   support

           0       0.89      0.81      0.85       481
           1       0.00      0.00      0.00        42
           2       0.93      0.98      0.95      1628

    accuracy                           0.92      2151
   macro avg       0.61      0.59      0.60      2151
weighted avg       0.90      0.92      0.91      2151


Best Parameters: OrderedDict([('max_depth', 42), ('max_features', 'sqrt'), ('min_samples_leaf', 3), ('min_samples_split', 8), ('n_estimators', 281)])

Accuracy: 0.9186
Elapsed Time: 34.17 s


RF + BO + SMOTE

In [ ]:
# BayesSearchCV dengan data hasil SMOTE
bayes_rf_smote = BayesSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    search_spaces=search_spaces_rf,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)

start = time.time()

bayes_rf_smote.fit(X_train_smote, y_train_smote)
pred_rf_bo_smote = bayes_rf_smote.predict(X_test_vec)

elapsed = time.time() - start

print("🔹 Bayesian Optimization (RF + BO + SMOTE)")
print(classification_report(y_test, pred_rf_bo_smote))
acc = accuracy_score(y_test, pred_rf_bo_smote)
print("Best Parameters:", bayes_rf_smote.best_params_)
print("\nAccuracy: {:.4f}".format(acc))
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 Bayesian Optimization (RF + BO + SMOTE)
              precision    recall  f1-score   support

           0       0.87      0.84      0.86       481
           1       0.15      0.45      0.23        42
           2       0.97      0.93      0.95      1628

    accuracy                           0.90      2151
   macro avg       0.67      0.74      0.68      2151
weighted avg       0.93      0.90      0.92      2151

Best Parameters: OrderedDict([('max_depth', 42), ('max_features', 'sqrt'), ('min_samples_leaf', 3), ('min_samples_split', 8), ('n_estimators', 281)])

Accuracy: 0.9024
Elapsed Time: 75.44 s


RF + Particle Swarm Optimization (PSO)

In [ ]:
def objective_rf(trial, X, y):
    model_params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 300),
        'max_depth': trial.suggest_int('max_depth', 10, 50),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
        'random_state': 42
    }

    model = RandomForestClassifier(**model_params)
    return cross_val_score(model, X, y, cv=5, scoring='accuracy').mean()

In [ ]:
# Jalankan optimasi
study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(lambda trial: objective_rf(trial, X_train_vec, y_train), n_trials=20)

# Train dan evaluasi model terbaik
best_params_rf = study_rf.best_params
model_rf = RandomForestClassifier(**best_params_rf, random_state=42)

start = time.time()

model_rf.fit(X_train_vec, y_train)
pred_rf_pso = model_rf.predict(X_test_vec)

elapsed = time.time() - start

print("\n🔹 Particle Swarm-like Optimization (RF + PSO)")
print(classification_report(y_test, pred_rf_pso))
acc = accuracy_score(y_test, pred_rf_pso)
print("\nBest Parameters:", best_params_rf)
print("\nAccuracy: {:.4f}".format(acc))
print("Elapsed Time: {:.2f} s".format(elapsed))

[I 2025-06-27 09:18:23,200] A new study created in memory with name: no-name-f816ad6f-8636-46ee-9297-2fd209d5e95e
[I 2025-06-27 09:18:25,369] Trial 0 finished with value: 0.8539705147088631 and parameters: {'n_estimators': 180, 'max_depth': 19, 'max_features': 'log2', 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.8539705147088631.
[I 2025-06-27 09:18:27,959] Trial 1 finished with value: 0.903499520289718 and parameters: {'n_estimators': 134, 'max_depth': 37, 'max_features': 'log2', 'min_samples_split': 10, 'min_samples_leaf': 1}. Best is trial 1 with value: 0.903499520289718.
[I 2025-06-27 09:18:34,765] Trial 2 finished with value: 0.9197754820750511 and parameters: {'n_estimators': 286, 'max_depth': 39, 'max_features': 'sqrt', 'min_samples_split': 7, 'min_samples_leaf': 4}. Best is trial 2 with value: 0.9197754820750511.
[I 2025-06-27 09:18:37,583] Trial 3 finished with value: 0.9214029836628246 and parameters: {'n_estimators': 120, 'max_depth': 50, 'ma


🔹 Particle Swarm-like Optimization (RF + PSO)
              precision    recall  f1-score   support

           0       0.90      0.82      0.85       481
           1       0.00      0.00      0.00        42
           2       0.93      0.98      0.95      1628

    accuracy                           0.92      2151
   macro avg       0.61      0.60      0.60      2151
weighted avg       0.90      0.92      0.91      2151


Best Parameters: {'n_estimators': 284, 'max_depth': 48, 'max_features': 'sqrt', 'min_samples_split': 9, 'min_samples_leaf': 1}

Accuracy: 0.9228
Elapsed Time: 2.45 s


RF + PSO + SMOTE

In [ ]:
study_rf_smote = optuna.create_study(direction='maximize')
study_rf_smote.optimize(lambda trial: objective_rf(trial, X_train_smote, y_train_smote), n_trials=20)

# Train dan evaluasi model terbaik
best_params_rf_smote = study_rf_smote.best_params
model_rf_smote = RandomForestClassifier(**best_params_rf_smote, random_state=42)

start = time.time()

model_rf_smote.fit(X_train_smote, y_train_smote)
pred_rf_pso_smote = model_rf_smote.predict(X_test_vec)

elapsed = time.time() - start

print("\n🔹 Particle Swarm-like Optimization (RF + PSO + SMOTE)")
print(classification_report(y_test, pred_rf_pso_smote))
acc = accuracy_score(y_test, pred_rf_pso_smote)
print("\nBest Parameters:", best_params_rf_smote)
print("\nAccuracy: {:.4f}".format(acc))
print("Elapsed Time: {:.2f} s".format(elapsed))

[I 2025-06-27 09:20:14,601] A new study created in memory with name: no-name-2ad3875f-df64-46cf-beee-914c9236c7fa
[I 2025-06-27 09:20:27,696] Trial 0 finished with value: 0.899795186891961 and parameters: {'n_estimators': 216, 'max_depth': 37, 'max_features': 'sqrt', 'min_samples_split': 8, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.899795186891961.
[I 2025-06-27 09:20:31,848] Trial 1 finished with value: 0.8689708141321045 and parameters: {'n_estimators': 173, 'max_depth': 29, 'max_features': 'log2', 'min_samples_split': 7, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.899795186891961.
[I 2025-06-27 09:20:39,794] Trial 2 finished with value: 0.8887864823348695 and parameters: {'n_estimators': 127, 'max_depth': 27, 'max_features': 'sqrt', 'min_samples_split': 10, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.899795186891961.
[I 2025-06-27 09:20:44,190] Trial 3 finished with value: 0.8582181259600615 and parameters: {'n_estimators': 256, 'max_depth': 11, 'max_

# **LOGISTIC REGRESSION**





In [20]:
# LR
lr = LogisticRegression(max_iter=200)
start = time.time()
lr.fit(X_train_vec, y_train)
pred_lr = lr.predict(X_test_vec)
elapsed = time.time() - start

print("🔹 Logistic Regression")
print(classification_report(y_test, pred_lr))
acc = accuracy_score(y_test, pred_lr)
print("\nAccuracy: {:.4f}".format(acc))
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 Logistic Regression
              precision    recall  f1-score   support

           0       0.93      0.88      0.90       481
           1       0.75      0.07      0.13        42
           2       0.95      0.99      0.97      1628

    accuracy                           0.94      2151
   macro avg       0.88      0.65      0.67      2151
weighted avg       0.94      0.94      0.94      2151


Accuracy: 0.9442
Elapsed Time: 0.12 s


LR + SMOTE

In [21]:
#LR+SMOTE
lr_smote = LogisticRegression(max_iter=200)
start = time.time()
lr_smote.fit(X_train_smote, y_train_smote)
pred_lr_smote = lr_smote.predict(X_test_vec)
elapsed = time.time() - start

print("🔹 Logistic Regression + SMOTE")
print(classification_report(y_test, pred_lr_smote))
acc = accuracy_score(y_test, pred_lr_smote)
print("\nAccuracy: {:.4f}".format(acc))
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 Logistic Regression + SMOTE
              precision    recall  f1-score   support

           0       0.88      0.91      0.90       481
           1       0.08      0.45      0.14        42
           2       0.99      0.87      0.93      1628

    accuracy                           0.87      2151
   macro avg       0.65      0.74      0.65      2151
weighted avg       0.95      0.87      0.90      2151


Accuracy: 0.8703
Elapsed Time: 0.24 s


In [22]:
#Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),
    ('clf', LogisticRegression(max_iter=200, solver='lbfgs', penalty='l2'))
])

In [23]:
#Pipeline SMOTE
pipeline_smote = ImbPipeline([
    ('tfidf', TfidfVectorizer(max_features=1000)),
    ('smote', SMOTE(random_state=42)),
    ('clf', LogisticRegression(max_iter=200, solver='lbfgs', penalty='l2'))
])

In [24]:
param_dist = {
    'tfidf__max_df': uniform_real(0.5, 0.5),
    'clf__C': uniform(0.01, 100)
}

LR + RS

In [25]:
#LR+RS
random_search_lr = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)

start = time.time()
random_search_lr.fit(X_train, y_train)
pred_lr_rs = random_search_lr.predict(X_test)
elapsed = time.time() - start

print("🔹 Logistic Regression + Random Search (C + max_df)")
print(classification_report(y_test, pred_lr_rs))
acc = accuracy_score(y_test, pred_lr_rs)
print("\nBest Parameters:")
print("C: {:.4f}".format(random_search_lr.best_params_['clf__C']))
print("max_df: {:.4f}".format(random_search_lr.best_params_['tfidf__max_df']))
print("Accuracy: {:.4f}".format(acc))
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 Logistic Regression + Random Search (C + max_df)
              precision    recall  f1-score   support

           0       0.92      0.88      0.90       481
           1       0.60      0.07      0.13        42
           2       0.95      0.99      0.97      1628

    accuracy                           0.94      2151
   macro avg       0.82      0.65      0.67      2151
weighted avg       0.94      0.94      0.94      2151


Best Parameters:
C: 2.0684
max_df: 0.9850
Accuracy: 0.9447
Elapsed Time: 17.59 s


LR + RS + SMOTE

In [26]:
#LR+ RS + SMOTE
random_search_lr_smote = RandomizedSearchCV(
    estimator=pipeline_smote,
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)
start = time.time()
random_search_lr_smote.fit(X_train, y_train)
pred_lr_rs_smote = random_search_lr_smote.predict(X_test)
elapsed = time.time() - start

print("🔹 Logistic Regression + Random Search + SMOTE (C + max_df)")
print(classification_report(y_test, pred_lr_rs_smote))
acc = accuracy_score(y_test, pred_lr_rs_smote)
print("\nBest Parameters:")
print("C: {:.4f}".format(random_search_lr_smote.best_params_['clf__C']))
print("max_df: {:.4f}".format(random_search_lr_smote.best_params_['tfidf__max_df']))
print("Accuracy: {:.4f}".format(acc))
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 Logistic Regression + Random Search + SMOTE (C + max_df)
              precision    recall  f1-score   support

           0       0.88      0.91      0.89       481
           1       0.07      0.33      0.12        42
           2       0.98      0.88      0.93      1628

    accuracy                           0.88      2151
   macro avg       0.65      0.71      0.65      2151
weighted avg       0.94      0.88      0.90      2151


Best Parameters:
C: 15.6119
max_df: 0.5780
Accuracy: 0.8763
Elapsed Time: 20.87 s


In [27]:
search_spaces_lr = {
    'tfidf__max_df': Real(0.5, 1.0),  # dari 0.5 s/d 1.0
    'clf__C': Real(0.01, 100, prior='log-uniform')
}

LR + BO

In [28]:
bayes_lr = BayesSearchCV(
    estimator=pipeline,
    search_spaces=search_spaces_lr,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)
start = time.time()
bayes_lr.fit(X_train, y_train)
pred_lr_bo = bayes_lr.predict(X_test)
elapsed = time.time() - start

print("🔹 Logistic Regression + Bayesian Optimization (C + max_df)")
print(classification_report(y_test, pred_lr_bo))
acc = accuracy_score(y_test, pred_lr_bo)
print("\nBest Parameters:")
print("C: {:.4f}".format(bayes_lr.best_params_['clf__C']))
print("max_df: {:.4f}".format(bayes_lr.best_params_['tfidf__max_df']))
print("Accuracy: {:.4f}".format(acc))
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 Logistic Regression + Bayesian Optimization (C + max_df)
              precision    recall  f1-score   support

           0       0.92      0.89      0.90       481
           1       0.67      0.10      0.17        42
           2       0.95      0.98      0.97      1628

    accuracy                           0.95      2151
   macro avg       0.85      0.66      0.68      2151
weighted avg       0.94      0.95      0.94      2151


Best Parameters:
C: 2.9398
max_df: 0.8877
Accuracy: 0.9451
Elapsed Time: 6.16 s


LR + BO + SMOTE

In [29]:
#LR+BO+SMOTE
bayes_lr_smote = BayesSearchCV(
    estimator=pipeline_smote,
    search_spaces=search_spaces_lr,
    n_iter=10,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)

start = time.time()
bayes_lr_smote.fit(X_train, y_train)
pred_lr_bo_smote = bayes_lr_smote.predict(X_test)
elapsed = time.time() - start

print("🔹 Logistic Regression + Bayesian Optimization + SMOTE (C + max_df)")
print(classification_report(y_test, pred_lr_bo_smote))
acc = accuracy_score(y_test, pred_lr_bo_smote)
print("\nBest Parameters:")
print("C: {:.4f}".format(bayes_lr_smote.best_params_['clf__C']))
print("max_df: {:.4f}".format(bayes_lr_smote.best_params_['tfidf__max_df']))
print("Accuracy: {:.4f}".format(acc))
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 Logistic Regression + Bayesian Optimization + SMOTE (C + max_df)
              precision    recall  f1-score   support

           0       0.88      0.90      0.89       481
           1       0.07      0.33      0.12        42
           2       0.98      0.88      0.93      1628

    accuracy                           0.88      2151
   macro avg       0.65      0.71      0.65      2151
weighted avg       0.94      0.88      0.91      2151


Best Parameters:
C: 17.7658
max_df: 0.5859
Accuracy: 0.8773
Elapsed Time: 16.35 s


LR + PSO

In [31]:
def objective_function(params):
    C_vals = params[:, 0]
    max_df_vals = params[:, 1]
    scores = []

    for C, max_df in zip(C_vals, max_df_vals):
        tfidf = TfidfVectorizer(max_features=1000, max_df=max_df)
        X_train_vec = tfidf.fit_transform(X_train)

        model = LogisticRegression(
            C=C,
            max_iter=200,
            penalty='l2',
            solver='lbfgs',
            fit_intercept=True
        )

        score = cross_val_score(model, X_train_vec, y_train, cv=5, scoring='accuracy').mean()
        scores.append(-score)
    return np.array(scores)
bounds = (np.array([0.01, 0.5]), np.array([5.0, 1.0]))
best_cost, best_pos = ps.single.GlobalBestPSO(
    n_particles=10,
    dimensions=2,
    options={'c1': 1.5, 'c2': 1.5, 'w': 0.8},
    bounds=bounds
).optimize(objective_function, iters=20)

best_C = best_pos[0]
best_max_df = best_pos[1]

2025-06-27 12:44:00,971 - pyswarms.single.global_best - INFO - Optimize for 20 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.8}
pyswarms.single.global_best: 100%|██████████|20/20, best_cost=-0.944
2025-06-27 12:46:36,119 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.9437264705484913, best pos: [4.65940365 0.58934124]


In [32]:
# LR+ PSO
tfidf_final = TfidfVectorizer(max_features=1000, max_df=best_max_df)
X_train_vec_pso = tfidf_final.fit_transform(X_train)
X_test_vec_pso = tfidf_final.transform(X_test)

start = time.time()
lr_pso = LogisticRegression(C=best_C, max_iter=200, solver='lbfgs', penalty='l2')
lr_pso.fit(X_train_vec_pso, y_train)
pred_lr_pso = lr_pso.predict(X_test_vec_pso)
elapsed = time.time() - start

print("🔹 Logistic Regression + PSO (C + max_df)")
print(classification_report(y_test, pred_lr_pso))
acc = accuracy_score(y_test, pred_lr_pso)
print("\nBest Parameters: C = {:.4f}, max_df = {:.4f}".format(best_C, best_max_df))
print("Accuracy: {:.4f}".format(acc))
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 Logistic Regression + PSO (C + max_df)
              precision    recall  f1-score   support

           0       0.91      0.89      0.90       481
           1       0.67      0.10      0.17        42
           2       0.96      0.98      0.97      1628

    accuracy                           0.95      2151
   macro avg       0.85      0.66      0.68      2151
weighted avg       0.94      0.95      0.94      2151


Best Parameters: C = 4.6594, max_df = 0.5893
Accuracy: 0.9461
Elapsed Time: 0.15 s


LR + PSO + SMOTE

In [33]:
def objective_function(params):
    C_vals = params[:, 0]
    max_df_vals = params[:, 1]
    scores = []

    for C, max_df in zip(C_vals, max_df_vals):
        tfidf = TfidfVectorizer(max_features=1000, max_df=max_df)
        X_vec = tfidf.fit_transform(X_train)

        smote = SMOTE(random_state=42)
        X_resampled, y_resampled = smote.fit_resample(X_vec, y_train)

        model = LogisticRegression(
            C=C,
            max_iter=200,
            penalty='l2',
            solver='lbfgs',
            fit_intercept=True
        )

        score = cross_val_score(model, X_resampled, y_resampled, cv=5, scoring='accuracy').mean()
        scores.append(-score)
    return np.array(scores)
bounds = (np.array([0.01, 0.5]), np.array([5.0, 1.0]))

best_cost, best_pos = ps.single.GlobalBestPSO(
    n_particles=10,
    dimensions=2,
    options={'c1': 1.5, 'c2': 1.5, 'w': 0.8},
    bounds=bounds
).optimize(objective_function, iters=20)

best_C = best_pos[0]
best_max_df = best_pos[1]

2025-06-27 12:46:51,110 - pyswarms.single.global_best - INFO - Optimize for 20 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.8}
pyswarms.single.global_best: 100%|██████████|20/20, best_cost=-0.916
2025-06-27 12:52:23,632 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.9160778289810547, best pos: [4.86848406 0.74542061]


In [34]:
#LR+PSO+SMOTE
tfidf_final = TfidfVectorizer(max_features=1000, max_df=best_max_df)
X_train_vec_final = tfidf_final.fit_transform(X_train)
X_test_vec_final = tfidf_final.transform(X_test)

# SMOTE
smote = SMOTE(random_state=42)
X_train_vec_smote_final, y_train_smote = smote.fit_resample(X_train_vec_final, y_train)

start = time.time()
lr_pso_smote = LogisticRegression(C=best_C, max_iter=200, solver='lbfgs', penalty='l2')
lr_pso_smote.fit(X_train_vec_smote_final, y_train_smote)
pred_lr_pso_smote = lr_pso_smote.predict(X_test_vec_final)
elapsed = time.time() - start

print("🔹 Logistic Regression + PSO + SMOTE")
print(classification_report(y_test, pred_lr_pso_smote))
acc = accuracy_score(y_test, pred_lr_pso_smote)
print("\nBest Parameters: C = {:.4f}, max_df = {:.4f}".format(best_C, best_max_df))
print("Accuracy: {:.4f}".format(acc))
print("Elapsed Time: {:.2f} s".format(elapsed))

🔹 Logistic Regression + PSO + SMOTE
              precision    recall  f1-score   support

           0       0.88      0.91      0.90       481
           1       0.07      0.36      0.12        42
           2       0.98      0.88      0.93      1628

    accuracy                           0.87      2151
   macro avg       0.65      0.71      0.65      2151
weighted avg       0.94      0.87      0.90      2151


Best Parameters: C = 4.8685, max_df = 0.7454
Accuracy: 0.8731
Elapsed Time: 0.36 s
